In [4]:
import sqlite3
import random
from faker import Faker

# Initialize Faker
fake = Faker()

# Connect to the SQLite database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Function to generate and insert data into the Customers table
def insert_customers(num_records):
    try:
        for _ in range(num_records):
            cursor.execute('''
                INSERT INTO Customers (FirstName, LastName, Email, Phone, Address, City, State, ZipCode, Country)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (fake.first_name(), fake.last_name(), fake.email(), fake.phone_number(), fake.address(), fake.city(), fake.state_abbr(), fake.zipcode(), fake.country()))
    except Exception as e:
        print(f"Error inserting customers: {e}")

# Function to generate and insert data into the ServiceCenters table
def insert_service_centers(num_records):
    try:
        for _ in range(num_records):
            cursor.execute('''
                INSERT INTO ServiceCenters (Name, Address, City, State, ZipCode, Country, Phone, Email)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            ''', (fake.company(), fake.address(), fake.city(), fake.state_abbr(), fake.zipcode(), fake.country(), fake.phone_number(), fake.email()))
    except Exception as e:
        print(f"Error inserting service centers: {e}")

# Function to generate and insert data into the Vehicles table
def insert_vehicles(num_records):
    try:
        customer_ids = [row[0] for row in cursor.execute('SELECT CustomerID FROM Customers').fetchall()]
        for _ in range(num_records):
            cursor.execute('''
                INSERT INTO Vehicles (CustomerID, Make, Model, Year, VIN, LicensePlate)
                VALUES (?, ?, ?, ?, ?, ?)
            ''', (random.choice(customer_ids), fake.company(), fake.word(), random.randint(1990, 2023), fake.uuid4(), fake.license_plate()))
    except Exception as e:
        print(f"Error inserting vehicles: {e}")

# Function to generate and insert data into the Services table
def insert_services():
    try:
        services = [
            ('Oil Change', 'Change the engine oil and replace the oil filter', 29.99),
            ('Tire Rotation', 'Rotate the tires to ensure even wear', 19.99),
            ('Brake Inspection', 'Inspect the brake system for wear and tear', 39.99),
            ('Battery Replacement', 'Replace the car battery', 89.99),
            ('Wheel Alignment', 'Align the wheels to ensure proper handling', 69.99)
        ]
        for service in services:
            cursor.execute('''
                INSERT INTO Services (ServiceName, Description, Price)
                VALUES (?, ?, ?)
            ''', service)
    except Exception as e:
        print(f"Error inserting services: {e}")

# Function to generate and insert data into the Mechanics table
def insert_mechanics(num_records):
    try:
        service_center_ids = [row[0] for row in cursor.execute('SELECT ServiceCenterID FROM ServiceCenters').fetchall()]
        for _ in range(num_records):
            cursor.execute('''
                INSERT INTO Mechanics (FirstName, LastName, Phone, Email, ServiceCenterID)
                VALUES (?, ?, ?, ?, ?)
            ''', (fake.first_name(), fake.last_name(), fake.phone_number(), fake.email(), random.choice(service_center_ids)))
    except Exception as e:
        print(f"Error inserting mechanics: {e}")

# Function to generate and insert data into the Parts table
def insert_parts(num_records):
    try:
        for _ in range(num_records):
            cursor.execute('''
                INSERT INTO Parts (PartName, Description, Price, StockQuantity)
                VALUES (?, ?, ?, ?)
            ''', (fake.word(), fake.sentence(), round(random.uniform(5.0, 500.0), 2), random.randint(1, 100)))
    except Exception as e:
        print(f"Error inserting parts: {e}")

# Function to generate and insert data into the Appointments table
def insert_appointments(num_records):
    try:
        customer_ids = [row[0] for row in cursor.execute('SELECT CustomerID FROM Customers').fetchall()]
        vehicle_ids = [row[0] for row in cursor.execute('SELECT VehicleID FROM Vehicles').fetchall()]
        service_center_ids = [row[0] for row in cursor.execute('SELECT ServiceCenterID FROM ServiceCenters').fetchall()]
        for _ in range(num_records):
            cursor.execute('''
                INSERT INTO Appointments (CustomerID, VehicleID, ServiceCenterID, AppointmentDate, Status)
                VALUES (?, ?, ?, ?, ?)
            ''', (random.choice(customer_ids), random.choice(vehicle_ids), random.choice(service_center_ids), fake.date_time_this_decade(), random.choice(['Scheduled', 'Completed', 'Cancelled'])))
    except Exception as e:
        print(f"Error inserting appointments: {e}")

# Function to generate and insert data into the AppointmentServices table
def insert_appointment_services(num_records):
    try:
        appointment_ids = [row[0] for row in cursor.execute('SELECT AppointmentID FROM Appointments').fetchall()]
        service_ids = [row[0] for row in cursor.execute('SELECT ServiceID FROM Services').fetchall()]
        for _ in range(num_records):
            cursor.execute('''
                INSERT INTO AppointmentServices (AppointmentID, ServiceID, Quantity)
                VALUES (?, ?, ?)
            ''', (random.choice(appointment_ids), random.choice(service_ids), random.randint(1, 5)))
    except Exception as e:
        print(f"Error inserting appointment services: {e}")

# Function to generate and insert data into the AppointmentParts table
def insert_appointment_parts(num_records):
    try:
        appointment_ids = [row[0] for row in cursor.execute('SELECT AppointmentID FROM Appointments').fetchall()]
        part_ids = [row[0] for row in cursor.execute('SELECT PartID FROM Parts').fetchall()]
        for _ in range(num_records):
            cursor.execute('''
                INSERT INTO AppointmentParts (AppointmentID, PartID, Quantity)
                VALUES (?, ?, ?)
            ''', (random.choice(appointment_ids), random.choice(part_ids), random.randint(1, 5)))
    except Exception as e:
        print(f"Error inserting appointment parts: {e}")

# Function to generate and insert data into the Invoices table
def insert_invoices(num_records):
    try:
        appointment_ids = [row[0] for row in cursor.execute('SELECT AppointmentID FROM Appointments').fetchall()]
        for _ in range(num_records):
            total_amount = round(random.uniform(50.0, 1000.0), 2)
            paid_amount = round(random.uniform(0.0, total_amount), 2)
            due_amount = total_amount - paid_amount
            cursor.execute('''
                INSERT INTO Invoices (AppointmentID, InvoiceDate, TotalAmount, PaidAmount, DueAmount)
                VALUES (?, ?, ?, ?, ?)
            ''', (random.choice(appointment_ids), fake.date_time_this_decade(), total_amount, paid_amount, due_amount))
    except Exception as e:
        print(f"Error inserting invoices: {e}")

# Function to generate and insert data into the Payments table
def insert_payments(num_records):
    try:
        invoice_ids = [row[0] for row in cursor.execute('SELECT InvoiceID FROM Invoices').fetchall()]
        for _ in range(num_records):
            cursor.execute('''
                INSERT INTO Payments (InvoiceID, PaymentDate, Amount, PaymentMethod)
                VALUES (?, ?, ?, ?)
            ''', (random.choice(invoice_ids), fake.date_time_this_decade(), round(random.uniform(10.0, 500.0), 2), random.choice(['Credit Card', 'Cash', 'Check', 'Bank Transfer'])))
    except Exception as e:
        print(f"Error inserting payments: {e}")

# Function to generate and insert data into the Reviews table
def insert_reviews(num_records):
    try:
        customer_ids = [row[0] for row in cursor.execute('SELECT CustomerID FROM Customers').fetchall()]
        service_center_ids = [row[0] for row in cursor.execute('SELECT ServiceCenterID FROM ServiceCenters').fetchall()]
        for _ in range(num_records):
            cursor.execute('''
                INSERT INTO Reviews (CustomerID, ServiceCenterID, Rating, Comments, ReviewDate)
                VALUES (?, ?, ?, ?, ?)
            ''', (random.choice(customer_ids), random.choice(service_center_ids), random.randint(1, 5), fake.sentence(), fake.date_time_this_decade()))
    except Exception as e:
        print(f"Error inserting reviews: {e}")

# Function to generate and insert data into the Employees table
def insert_employees(num_records):
    try:
        service_center_ids = [row[0] for row in cursor.execute('SELECT ServiceCenterID FROM ServiceCenters').fetchall()]
        for _ in range(num_records):
            cursor.execute('''
                INSERT INTO Employees (FirstName, LastName, Email, Phone, Position, ServiceCenterID)
                VALUES (?, ?, ?, ?, ?, ?)
            ''', (fake.first_name(), fake.last_name(), fake.email(), fake.phone_number(), fake.job(), random.choice(service_center_ids)))
    except Exception as e:
        print(f"Error inserting employees: {e}")

# Function to generate and insert data into the EmployeeSchedules table
def insert_employee_schedules(num_records):
    try:
        employee_ids = [row[0] for row in cursor.execute('SELECT EmployeeID FROM Employees').fetchall()]
        for _ in range(num_records):
            cursor.execute('''
                INSERT INTO EmployeeSchedules (EmployeeID, WorkDate, StartTime, EndTime)
                VALUES (?, ?, ?, ?)
            ''', (random.choice(employee_ids), fake.date_this_year(), fake.time(), fake.time()))
    except Exception as e:
        print(f"Error inserting employee schedules: {e}")

# Function to generate and insert data into the Inventory table
def insert_inventory(num_records):
    try:
        part_ids = [row[0] for row in cursor.execute('SELECT PartID FROM Parts').fetchall()]
        service_center_ids = [row[0] for row in cursor.execute('SELECT ServiceCenterID FROM ServiceCenters').fetchall()]
        for _ in range(num_records):
            cursor.execute('''
                INSERT INTO Inventory (PartID, ServiceCenterID, StockQuantity)
                VALUES (?, ?, ?)
            ''', (random.choice(part_ids), random.choice(service_center_ids), random.randint(1, 100)))
    except Exception as e:
        print(f"Error inserting inventory: {e}")

# Function to generate and insert data into the Suppliers table
def insert_suppliers(num_records):
    try:
        for _ in range(num_records):
            cursor.execute('''
                INSERT INTO Suppliers (Name, ContactName, Phone, Email, Address, City, State, ZipCode, Country)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (fake.company(), fake.name(), fake.phone_number(), fake.email(), fake.address(), fake.city(), fake.state_abbr(), fake.zipcode(), fake.country()))
    except Exception as e:
        print(f"Error inserting suppliers: {e}")

# Function to generate and insert data into the Orders table
def insert_orders(num_records):
    try:
        supplier_ids = [row[0] for row in cursor.execute('SELECT SupplierID FROM Suppliers').fetchall()]
        for _ in range(num_records):
            cursor.execute('''
                INSERT INTO Orders (SupplierID, OrderDate, TotalAmount, Status)
                VALUES (?, ?, ?, ?)
            ''', (random.choice(supplier_ids), fake.date_time_this_decade(), round(random.uniform(100.0, 10000.0), 2), random.choice(['Pending', 'Completed', 'Cancelled'])))
    except Exception as e:
        print(f"Error inserting orders: {e}")

# Function to generate and insert data into the OrderDetails table
def insert_order_details(num_records):
    try:
        order_ids = [row[0] for row in cursor.execute('SELECT OrderID FROM Orders').fetchall()]
        part_ids = [row[0] for row in cursor.execute('SELECT PartID FROM Parts').fetchall()]
        for _ in range(num_records):
            cursor.execute('''
                INSERT INTO OrderDetails (OrderID, PartID, Quantity, Price)
                VALUES (?, ?, ?, ?)
            ''', (random.choice(order_ids), random.choice(part_ids), random.randint(1, 100), round(random.uniform(5.0, 500.0), 2)))
    except Exception as e:
        print(f"Error inserting order details: {e}")

# Function to generate and insert data into the Warranties table
def insert_warranties(num_records):
    try:
        vehicle_ids = [row[0] for row in cursor.execute('SELECT VehicleID FROM Vehicles').fetchall()]
        for _ in range(num_records):
            cursor.execute('''
                INSERT INTO Warranties (VehicleID, WarrantyProvider, StartDate, EndDate, CoverageDetails)
                VALUES (?, ?, ?, ?, ?)
            ''', (random.choice(vehicle_ids), fake.company(), fake.date_this_decade(), fake.date_this_decade(), fake.sentence()))
    except Exception as e:
        print(f"Error inserting warranties: {e}")

# Insert data into tables
insert_customers(1000000)
conn.commit()
insert_service_centers(500)
conn.commit()
insert_vehicles(200000)
conn.commit()
insert_services()
conn.commit()
insert_mechanics(100000)
conn.commit()
insert_parts(50000)
conn.commit()
insert_appointments(800000)
conn.commit()
insert_appointment_services(100000)
conn.commit()
insert_appointment_parts(10000)
conn.commit()
insert_invoices(1000000)
conn.commit()
insert_payments(2000000)
conn.commit()
insert_reviews(100000)
conn.commit()
insert_employees(5000)
conn.commit()
insert_employee_schedules(2000000)
conn.commit()
insert_inventory(2000000)
conn.commit()
insert_suppliers(20000)
conn.commit()
insert_orders(500000)
conn.commit()
insert_order_details(1000000)
conn.commit()
insert_warranties(5000000)
conn.commit()
# Close the connection
conn.close()

Error inserting customers: UNIQUE constraint failed: Customers.Email
Error inserting employees: UNIQUE constraint failed: Employees.Email
